In [ ]:
import csv
import numpy as np
import os

This file is similar to the data_processing_gazecom.ipynb file as we precompute the windows for the hollywood dataset. The differences are due to different file format and that wie do not perform data augmentation.

In [ ]:
def load_data(path):
    for root, dirs, files in os.walk(path):
        
        if files:
            for file in files:
                data_set = []
                if file.lower().endswith('.coord'):
                    with open(root + '/' + file) as csvfile:
                        data = csv.reader(csvfile, delimiter=' ')
                        data = list(list(rec) for rec in data)
                    i = 0
                    while i < len(data):
                        if data[i][0] == '#':
                            #skip unwanted information
                            i += 1
                        elif data[i][0] == 'gaze':
                            #save screen resolution
                            x_res = float(data[i][1])
                            y_res = float(data[i][2])
                            i += 1
                        elif data[i][0] == 'geometry':
                            #skip unwanted information
                            i += 1
                        else:
                            norm_x = float(data[i][1])/x_res
                            norm_y = float(data[i][2])/y_res
                            data_set.append([norm_x,norm_y])
                            #skip three samples to downsample from 1000hz to 250hz
                            i += 4

                    samples = process_data(data_set, window_size=251)
                    samples = np.asarray(samples)
                    np.save('/bigpool/strohmfn/hollywood_processed/' + file + ".npy", samples)
    return data_set

In [ ]:
def create_training_sample(data_set, window_size, center_index):
    sample = []
    for i in range(center_index-int(window_size/2), center_index+int(window_size/2)+1):
        d = data_set[i][0:2]
        sample = sample + [d]
    return sample

In [ ]:
def process_data(data_set, window_size):
    samples = []

    for i in range(int(window_size/2), len(data_set)-int(window_size/2)):
        sample = create_training_sample(data_set, window_size, i)
        samples.append(sample)
    return samples

In [ ]:
load_data('/bigpool/strohmfn/hollywood')